In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    DDD_Train_Decile5 = pd.read_pickle("DDD_Train_Decile5.pkl")
    DDD_Train_Targets_1D = pd.read_pickle("DDD_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("DDD_Test_Decile5.pkl")
    y_test = pd.read_pickle("DDD_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(DDD_Train_Decile5, DDD_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150,200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM DDD Decile 5')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

2264/2264 [==============================] - 5s 2ms/step - loss: 0.4262 - acc: 0.8269 - val_loss: 0.3991 - val_acc: 0.8377
Epoch 39/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4204 - acc: 0.8255 - val_loss: 0.3989 - val_acc: 0.8377
Epoch 40/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4195 - acc: 0.8264 - val_loss: 0.4047 - val_acc: 0.8342
Epoch 41/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4208 - acc: 0.8286 - val_loss: 0.3999 - val_acc: 0.8360
Epoch 42/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4207 - acc: 0.8216 - val_loss: 0.3940 - val_acc: 0.8413
Epoch 43/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4196 - acc: 0.8357 - val_loss: 0.3915 - val_acc: 0.8430
Epoch 44/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4156 - acc: 0.8295 - val_loss: 0.3874 - val_acc: 0.8430
Epoch 45/100
2264/2264 [==============================] - 5s 

Epoch 98/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4037 - acc: 0.8357 - val_loss: 0.3696 - val_acc: 0.8448
Epoch 99/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4045 - acc: 0.8339 - val_loss: 0.3691 - val_acc: 0.8448
Epoch 100/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4020 - acc: 0.8295 - val_loss: 0.3698 - val_acc: 0.8466
Test accuracy: 0.8465608462454781
Train on 2264 samples, validate on 567 samples
Epoch 1/150
2264/2264 [==============================] - 7s 3ms/step - loss: 0.6434 - acc: 0.5866 - val_loss: 0.5841 - val_acc: 0.6402
Epoch 2/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.5381 - acc: 0.6952 - val_loss: 0.4896 - val_acc: 0.7160
Epoch 3/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.5052 - acc: 0.7743 - val_loss: 0.4588 - val_acc: 0.8078
Epoch 4/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4701 - acc: 0.8105 - val_l

2264/2264 [==============================] - 5s 2ms/step - loss: 0.3776 - acc: 0.8330 - val_loss: 0.3488 - val_acc: 0.8466
Epoch 57/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3695 - acc: 0.8326 - val_loss: 0.3488 - val_acc: 0.8466
Epoch 58/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3775 - acc: 0.8397 - val_loss: 0.3490 - val_acc: 0.8466
Epoch 59/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3781 - acc: 0.8317 - val_loss: 0.3489 - val_acc: 0.8466
Epoch 60/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3751 - acc: 0.8348 - val_loss: 0.3489 - val_acc: 0.8466
Epoch 61/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3707 - acc: 0.8357 - val_loss: 0.3486 - val_acc: 0.8466
Epoch 62/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3736 - acc: 0.8326 - val_loss: 0.3484 - val_acc: 0.8466
Epoch 63/150
2264/2264 [==============================] - 5s 

2264/2264 [==============================] - 5s 2ms/step - loss: 0.3776 - acc: 0.8366 - val_loss: 0.3486 - val_acc: 0.8466
Epoch 117/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3800 - acc: 0.8383 - val_loss: 0.3488 - val_acc: 0.8466
Epoch 118/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3824 - acc: 0.8405 - val_loss: 0.3488 - val_acc: 0.8466
Epoch 119/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3746 - acc: 0.8458 - val_loss: 0.3488 - val_acc: 0.8466
Epoch 120/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3749 - acc: 0.8348 - val_loss: 0.3488 - val_acc: 0.8466
Epoch 121/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3785 - acc: 0.8383 - val_loss: 0.3486 - val_acc: 0.8466
Epoch 122/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3809 - acc: 0.8352 - val_loss: 0.3490 - val_acc: 0.8466
Epoch 123/150
2264/2264 [==============================

...

2264/2264 [==============================] - 5s 2ms/step - loss: 0.3969 - acc: 0.8295 - val_loss: 0.3675 - val_acc: 0.8519
Epoch 66/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3931 - acc: 0.8260 - val_loss: 0.3666 - val_acc: 0.8554
Epoch 67/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3975 - acc: 0.8269 - val_loss: 0.3669 - val_acc: 0.8483
Epoch 68/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4086 - acc: 0.8308 - val_loss: 0.3677 - val_acc: 0.8466
Epoch 69/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3971 - acc: 0.8313 - val_loss: 0.3641 - val_acc: 0.8501
Epoch 70/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3981 - acc: 0.8299 - val_loss: 0.3672 - val_acc: 0.8466
Epoch 71/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3975 - acc: 0.8255 - val_loss: 0.3640 - val_acc: 0.8571
Epoch 72/100
2264/2264 [==============================] - 5s 


Epoch 00023: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
Epoch 24/200
2264/2264 [==============================] - 3s 1ms/step - loss: 0.3779 - acc: 0.8370 - val_loss: 0.3530 - val_acc: 0.8413
Epoch 25/200
2264/2264 [==============================] - 3s 1ms/step - loss: 0.3789 - acc: 0.8339 - val_loss: 0.3530 - val_acc: 0.8413
Epoch 26/200
2264/2264 [==============================] - 3s 1ms/step - loss: 0.3786 - acc: 0.8344 - val_loss: 0.3528 - val_acc: 0.8413
Epoch 27/200
2264/2264 [==============================] - 3s 1ms/step - loss: 0.3796 - acc: 0.8370 - val_loss: 0.3528 - val_acc: 0.8395
Epoch 28/200
2264/2264 [==============================] - 3s 1ms/step - loss: 0.3846 - acc: 0.8335 - val_loss: 0.3529 - val_acc: 0.8395

Epoch 00028: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 29/200
2264/2264 [==============================] - 3s 1ms/step - loss: 0.3866 - acc: 0.8291 - val_loss: 0.3528 - val_acc: 0.8413
Epoch 30/200
2264/2264 [=============

2264/2264 [==============================] - 3s 1ms/step - loss: 0.3789 - acc: 0.8277 - val_loss: 0.3525 - val_acc: 0.8377
Epoch 84/200
2264/2264 [==============================] - 3s 1ms/step - loss: 0.3771 - acc: 0.8322 - val_loss: 0.3526 - val_acc: 0.8377
Epoch 85/200
2264/2264 [==============================] - 3s 1ms/step - loss: 0.3846 - acc: 0.8352 - val_loss: 0.3525 - val_acc: 0.8377
Epoch 86/200
2264/2264 [==============================] - 3s 1ms/step - loss: 0.3795 - acc: 0.8335 - val_loss: 0.3525 - val_acc: 0.8377
Epoch 87/200
2264/2264 [==============================] - 3s 1ms/step - loss: 0.3757 - acc: 0.8375 - val_loss: 0.3524 - val_acc: 0.8377
Epoch 88/200
2264/2264 [==============================] - 3s 1ms/step - loss: 0.3829 - acc: 0.8335 - val_loss: 0.3526 - val_acc: 0.8395
Epoch 89/200
2264/2264 [==============================] - 3s 1ms/step - loss: 0.3770 - acc: 0.8344 - val_loss: 0.3525 - val_acc: 0.8377
Epoch 90/200
2264/2264 [==============================] - 3s 

2264/2264 [==============================] - 3s 1ms/step - loss: 0.3800 - acc: 0.8379 - val_loss: 0.3521 - val_acc: 0.8360
Epoch 144/200
2264/2264 [==============================] - 3s 1ms/step - loss: 0.3810 - acc: 0.8379 - val_loss: 0.3520 - val_acc: 0.8342
Epoch 145/200
2264/2264 [==============================] - 3s 1ms/step - loss: 0.3824 - acc: 0.8370 - val_loss: 0.3520 - val_acc: 0.8342
Epoch 146/200
2264/2264 [==============================] - 3s 1ms/step - loss: 0.3757 - acc: 0.8357 - val_loss: 0.3521 - val_acc: 0.8360
Epoch 147/200
2264/2264 [==============================] - 3s 1ms/step - loss: 0.3796 - acc: 0.8344 - val_loss: 0.3521 - val_acc: 0.8377
Epoch 148/200
2264/2264 [==============================] - 3s 1ms/step - loss: 0.3801 - acc: 0.8339 - val_loss: 0.3521 - val_acc: 0.8377
Epoch 149/200
2264/2264 [==============================] - 3s 1ms/step - loss: 0.3808 - acc: 0.8348 - val_loss: 0.3521 - val_acc: 0.8377
Epoch 150/200
2264/2264 [==============================

2264/2264 [==============================] - 3s 1ms/step - loss: 0.5810 - acc: 0.6674 - val_loss: 0.5478 - val_acc: 0.6843
Epoch 3/50
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5259 - acc: 0.7297 - val_loss: 0.4784 - val_acc: 0.7513
Epoch 4/50
2264/2264 [==============================] - 3s 1ms/step - loss: 0.4971 - acc: 0.7862 - val_loss: 0.4599 - val_acc: 0.8113
Epoch 5/50
2264/2264 [==============================] - 3s 1ms/step - loss: 0.4866 - acc: 0.8057 - val_loss: 0.4462 - val_acc: 0.8113
Epoch 6/50
2264/2264 [==============================] - 3s 1ms/step - loss: 0.4663 - acc: 0.8233 - val_loss: 0.4301 - val_acc: 0.8236
Epoch 7/50
2264/2264 [==============================] - 3s 1ms/step - loss: 0.4519 - acc: 0.8269 - val_loss: 0.4382 - val_acc: 0.8095
Epoch 8/50
2264/2264 [==============================] - 3s 1ms/step - loss: 0.4434 - acc: 0.8260 - val_loss: 0.4053 - val_acc: 0.8254
Epoch 9/50
2264/2264 [==============================] - 3s 1ms/step - los

In [ ]:
0.86434